# An Improved Symmetric Homomorphic Encryption Scheme

In this notebook, a symetric somewhat homomorphic encryption system is introduced that uses a sequence of singular linear transforms to make known plaintext attacks impossible, and output a  ciphertext of size greater than or equal to the plaintext size.

## Introduction
Homomorphic encryption differs from conventional symmetric or asymmetric encryption schemes such as AES or RSA, in that it allows for computation in the encrypted domain.  Conventionally, when encrypting data with with AES for example, after the data has been encrypted, there is nothing that can be done with the data aside from storing it in encrypted form or sending it to another party.  If the encrypted data (ciphertext) was to be operated on in some capacity, such as through multiplication, it would not decrypt correctly, resulting in "garbage".  Conversely, if the plaintext data was to be encrypted homomorphically, the ciphertext could be operated on and subsequently decrypted correctly.

For example, suppose the number 6 was sensitive information that Alice could not reveal to Bob, however Bob was the only party available that could perform the necessary computations on Alice's information.  Alice could cirvumvent this issue with homomorphic encryption.  She could encrypt 6 with a homomorphic encryption scheme, and send it to Bob.  Bob would then perform on operation on it, say multiplying it by 13, all the while not knowing what exactly he is operating on as Alice's data is in encrypted form.  When alice receives the operated ciphertext from Bob, she will decrypt it to find her original data, 6, was converted into the number 78, the product of 6 and 13.

#### Practical Example of Homomorphic Encryption
Homomorphic encryption has many exciting applications in the practical domain.  One example is that of encrypted machine learning.  Assume Google has developed a novel recurrent neural network they want to use for word prediction in Android users keyboards.  Google could take the Androids users data, storing it on a centralized server and train the model there.  This option would be fast, however it has the massive drawback of pulling the users data plaintext into a centralized location, not the best option in terms of user privacy.  Google would also have the option of useing the technique of federated learning, to send the neural network to each user individually, training the network locally on each users device, and then pulling these "child networks" together, synthesising their gradients to produce the parent network that would result in the final product.  Unfortunately, this recurrent neural network is sensitive intellectual property, and Google couldn't risk revealing this network to the competition.  

Google can fix this problem with homomorphic encryption.  Google could encrypt the neural network homomorphically, sending the encrypted model down to each Android users device.  Because this model is encrypted homomorphically, it could be trained securely, locally on each device, and when finished training, could be sent back to Google to be decrypted and finished.  

## Homomorphic Encryption Scheme
Following is a préci of the mathematics of homomorphic encryption, specifically as it applies to the scheme posited in this notebook.  The basic homomorphic encryption scheme as posited in the paper Efficient Homomorphic Encryption on Integer Vectors and Its Applicationsis by Zhou et al [1] is as follows.  

Given a plaintext vector $\mathbf{x} \in \mathbb{Z}^n$, and a nonsingular secret key $S \in \mathbb{Z}^{nxn}$, $\mathbf{c} \in \mathbb{Z}^n$ is a vector that satisfies 

$$
S \mathbf{c} = w \mathbf{x} + \mathbf{e}
$$

where $w$ is a large integer and $\mathbf{e}$ is an error vector.  The scalar value $w$ is used to scale up the plaintext vector $\mathbf{x}$ before encryption to prevent the plaintext from succumbing to noise during operations in the ciphertext domain.  The vector $\mathbf{e}$ is a noise vector used to make adversarial attacks on the ciphertext more difficult.  The decryption scheme is as follows.

$$
\mathbf{x} = \lceil \frac{S \mathbf{c}}{w} \rfloor
$$

Note how the $\mathbf{e}$ vector is not needed in decryption as it as shrugged off as rounding error.  The flaws of this homomorphic encryption scheme are apparent.  First, because the secret key $S$ is nonsingular, the plaintext and ciphertext are of the same dimension, allowing for leaks to be taken advantage of in its cryptanalysis.  Secondly, given a set of plaintext/ciphertext pairs, $S$ can be solved for in a plaintext attack as shown in the paper Cryptanalysis of a Homomorphic Encryption Scheme by Bogos et al [2].  

In the rest of the notebook, a somewhat homomorphic encryption scheme is proposed that addresses the two problems outlined above.  Problem one, of the domain and codomain of the encryption scheme being the same size is solved through the use of singular $m \times n$ matrices where $m >= n$.  The inverse of these matrices for decryption purposes can be found with the left inverse as follows

$$
A^{-1}_{\text{left}} = (A^T A)^{-1} A^T
$$

Problem two, being the standard schemes susceptibility to known plaintext attacks is solved by passing the plaintext through a sequence of linear and nonlinear transformations.  The plaintext is first passed through a linear transformation, followed by a nonlinear transformation, followed by one more linear transformation.  Unfortunately this sequence of operations is complex and slow, however it does prevent any form of plaintext attack on the cipher.

## The Program
Below, the improved cipher is implemented in code, and tested in many different situations.  See the comments in each code block for line to line details.  

Begin by importing libraries, setting up keys and defining functions.

In [58]:
import numpy as np

# Define the plaintext message that will be encrypted and operated on in the encrypted domain
# for the purpose of this example, the elements of the message vector are simply random integers
message = np.array([9.0, 4.0, 2.0, 6.0, 8.0, 15.0, 0.0, 1.0])

key1 = np.random.randint(1, 30, ((len(message) + np.random.randint(0, 10)), len(message)))
key2 = np.random.randint(1, 30, ((np.random.randint(0, 4) + key1.shape[0]), key1.shape[0]))

In [59]:
def encrypt(message, key1, key2):
    # scale up plaintext to avoid noise damage
    message *= 20
    
    ciphertext = np.matmul(key1, message)
    # pass through nonlinear function
    ciphertext = (2 * ciphertext) + 27
    ciphertext = np.matmul(key2, ciphertext)
    # add noise vector to ciphertext
    e = np.random.rand(len(ciphertext))
    ciphertext += e
    return ciphertext

In [60]:
# define a rounding function for use in decryption
def myround(a, decimals=0):
     return np.around(a-10**(-(decimals+5)), decimals=decimals)
    
# make decryption function, just inverse order of encryption function
def decrypt(ciphertext, key1, key2):
    key1inv = np.matmul((np.linalg.inv(np.matmul(key1.T, key1))), key1.T)
    key2inv = np.matmul((np.linalg.inv(np.matmul(key2.T, key2))), key2.T)
    
    plaintext = np.matmul(ciphertext, key2inv.T)
    plaintext = (0.5 * plaintext) - 27
    plaintext = np.matmul(plaintext, key1inv.T)
    plaintext *= (1/20)
    return plaintext

## Testing
Below tests will be run with the encryption scheme.  We will start by just encrypting and decrypting the plaintext to make sure everything is working ok.

In [61]:
# encrypt
ciphertext = encrypt(message, key1, key2)
print("The ciphertext is: \n\n", ciphertext)

# decrypt
plaintext = decrypt(ciphertext, key1, key2)
plaintext = myround(plaintext)
print("\n\nThe plaintext is \n\n", plaintext)

The ciphertext is: 

 [7554969.231594   6637650.61629228 6386368.92248635 8658316.28469172
 7405603.71482476 8066256.84235775 6532414.39939703 7018361.40482091
 9253533.09353412 6804317.45193388 8290949.14436013 6736317.23637433
 5637116.55677342 5380742.51766601 8763250.38124634 8080573.44252275
 6754787.21861457 7429019.73355288]


The plaintext is 

 [ 9.  4.  2.  6.  8. 15. -0.  1.]


Now that we know the encryption scheme is working, operations can be performed on the ciphertext to show the power of homomorphic encryption.  

In [72]:
# redefine arguments
message = np.array([9.0, 4.0, 2.0, 6.0, 8.0, 15.0, 0.0, 1.0])

key1 = np.random.randint(1, 30, ((len(message) + np.random.randint(0, 10)), len(message)))
key2 = np.random.randint(1, 30, ((np.random.randint(0, 4) + key1.shape[0]), key1.shape[0]))

# encrypt
ciphertext2= encrypt(message, key1, key2)

# operate on ciphertext
ciphertext2 *= 6
ciphertext2 *= 3
ciphertext2 = ciphertext2 + ciphertext2

print("The ciphertext is: \n\n", ciphertext2)

# now decrypt
plaintext2 = decrypt(ciphertext2, key1, key2)
plaintext2 = myround(plaintext2)

print("\n\nThe plaintext is \n\n", plaintext2)

The ciphertext is: 

 [1.92934304e+08 1.87674807e+08 2.06553214e+08 2.13532094e+08
 2.32749289e+08 2.23001926e+08 1.97442378e+08 2.14514692e+08
 1.96917749e+08 2.33360831e+08 2.74791841e+08 1.81527766e+08
 1.97327182e+08 2.41648716e+08 2.43549912e+08 1.40447991e+08
 2.60969196e+08]


The plaintext is 

 [324. 144.  72. 216. 288. 540.   0.  36.]


## Results
As demonstrated in the program above, the improved homomorphic encryption scheme works beautifully.  The ciphertext is of a size larger than the plaintext, and operations can be performed on the ciphertext, and the ciphertext will decrypt properly.  

In the example above, the ciphertext was first multiplied by 6, then multiplied by 3, then added to itself and it decrypted to the correct value for each element of the plaintext (from 9 to 324, 4 to 144 etc.).  

Unfortunately, furthur tests with the current cipher show it cannot yet earn the title of being a "fully homomorphic encryption scheme".  This is because, as more and more operations are stacked ontop of eachother, the noise corrupts enough of the signal to output incorrect plaintext, so the scheme currently falls under the category of being a "somewhat homomorphic encryption scheme".

## Improvements
This cipher is still in its infancy, and has much room for improvement.  In the following list, I propose some potential for the cipher.
* One major improvement would be allowing for the generation of a public/private key pair as is implemented in the referenced papers.
* This cipher does not currently allow for an arbitrary number of operations to be performed on the ciphertext while still maintaining its correctness.  Too many addition/multiplication operations on the ciphertext causes noise buildup, resulting in incorrect decrypted plaintext.
* The encryption scheme is currently somewhat homomorphic, and should be improved to earn the title of fully homomorphic.

## Bibliography
1. [Efficient Homomorphic Encryption on Integer Vectors and Its Applications](http://allegro.mit.edu/pubs/posted/conference/2014-zhou-wornell-ita.pdf), Zhou et al
2. [Cryptanalysis of a Homomorphic Encryption Scheme](https://eprint.iacr.org/2016/775.pdf), Bogos et al
3. [Efficient Integer Vector Homomorphic Encryption](http://courses.csail.mit.edu/6.857/2015/files/yu-lai-payor.pdf), Yu et al